In [1]:
# Dependencies

import pandas as pd
import geopandas as gpd
import numpy as np  

#### Get Total Days Per Year

In [13]:
total_days = raw_stats.groupby(['year', 'ID_HDC_G0'])['duration'].sum()

In [16]:
total_days_df = pd.DataFrame(total_days).reset_index()
total_days_df.rename({'duration': 'total_days'}, axis=1, inplace=True)

In [17]:
total_days_df

,year,ID_HDC_G0,total_days
0,1983,28,1
1,1983,29,2
2,1983,33,1
3,1983,34,2
4,1983,35,2
...,...,...,...
153512,2016,13024,1
153513,2016,13025,1
153514,2016,13027,1
153515,2016,13041,2


In [23]:
def make_pdays(df_stats, df_pop, scale):
    
    """  Makes a dataframe with Tmax stats and population to calc people days.
    
    Args: 
        df_stats = Tmax stats output
        df_pop = interpolated GHS-UCDB population
        scale = if you want to divide the data ... 10**9 is best for global scale
    """
    
 
    # Make Population Long Format
    pop_long = pd.wide_to_long(df_pop, stubnames = 'P', i = 'ID_HDC_G0', j = 'year')
    pop_long.reset_index(level=0, inplace=True)
    pop_long.reset_index(level=0, inplace=True)
    pop_long = pop_long.drop('Unnamed: 0', axis = 1)
    
    # Get Total Days 
    data = df_stats
    pdays = pd.DataFrame()
    pdays['ID_HDC_G0'] = data['ID_HDC_G0']
    pdays['year'] = data['year']
    pdays['total_days'] = data['total_days']

    # Merge
    pdays_merge = pdays.merge(pop_long, on=['ID_HDC_G0', 'year'], how = 'left')

    # Now get people days from 1983 and change
    p = pd.DataFrame()
    p['ID_HDC_G0'] = df_pop['ID_HDC_G0']
    p['P1983'] = df_pop['P1983']
    p['P2016'] = df_pop['P2016']

    pdays_merge = pdays_merge.merge(p ,on=['ID_HDC_G0'], how = 'left')
    
    # Calc p days = total days i * pop i 
    pdays_merge['people_days'] = pdays_merge['total_days'] * pdays_merge['P'] / scale # total people days
    
    # Pdays due to heat increase = total days i - pop 83 - (total days 83 - pop 83) <<-- back out when graphing
    pdays_merge['people_days_heat'] = pdays_merge['total_days'] * pdays_merge['P1983'] / scale # people days w/ pop con
    
    # Pdays due to pop increase = total days i * (pop i - pop 83)
    pdays_merge['people_days_pop'] = pdays_merge['total_days'] *(pdays_merge['P'] - pdays_merge['P1983']) / scale # dif

    return pdays_merge

In [24]:
test = make_pdays(total_days_df, df_pop)

In [26]:
test[test['ID_HDC_G0'] == 28]

,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.000000e+00
4483,28,1984,1,59854.153489,59320.971209,91449.606255,0.000060,0.000059,5.331823e-07
21263,28,1988,4,61986.882611,59320.971209,91449.606255,0.000248,0.000237,1.066365e-05
25823,28,1989,1,62520.064891,59320.971209,91449.606255,0.000063,0.000059,3.199094e-06
30132,28,1990,5,63053.247172,59320.971209,91449.606255,0.000315,0.000297,1.866138e-05
50664,28,1995,1,66183.750420,59320.971209,91449.606255,0.000066,0.000059,6.862779e-06
59549,28,1997,3,67435.951719,59320.971209,91449.606255,0.000202,0.000178,2.434494e-05
63898,28,1998,5,68062.052369,59320.971209,91449.606255,0.000340,0.000297,4.370541e-05
68826,28,1999,3,68688.153018,59320.971209,91449.606255,0.000206,0.000178,2.810155e-05
73414,28,2000,4,69314.253668,59320.971209,91449.606255,0.000277,0.000237,3.997313e-05


In [29]:
b


In [30]:
test2 = add_years(test)

In [33]:
test2[0]

,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0
1,29,1983,2,336518.836621,336518.836621,533318.453653,0.000673,0.000673,0.0
2,33,1983,1,40107.651544,40107.651544,61668.587152,0.000040,0.000040,0.0
3,34,1983,2,236515.041712,236515.041712,365617.969896,0.000473,0.000473,0.0
4,35,1983,2,224762.325023,224762.325023,297123.339094,0.000450,0.000450,0.0
...,...,...,...,...,...,...,...,...,...
85430,13076,2011,0,936431.423297,645048.225121,990070.336660,0.000000,0.000000,0.0
85431,13076,2012,0,949841.151638,645048.225121,990070.336660,0.000000,0.000000,0.0
85432,13076,2013,0,963250.879978,645048.225121,990070.336660,0.000000,0.000000,0.0
85433,13076,2015,0,990070.336660,645048.225121,990070.336660,0.000000,0.000000,0.0


In [34]:
test2[0][test2[0]['ID_HDC_G0'] == 28]

,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.000000e+00
4483,28,1984,1,59854.153489,59320.971209,91449.606255,0.000060,0.000059,5.331823e-07
21263,28,1988,4,61986.882611,59320.971209,91449.606255,0.000248,0.000237,1.066365e-05
25823,28,1989,1,62520.064891,59320.971209,91449.606255,0.000063,0.000059,3.199094e-06
30132,28,1990,5,63053.247172,59320.971209,91449.606255,0.000315,0.000297,1.866138e-05
50664,28,1995,1,66183.750420,59320.971209,91449.606255,0.000066,0.000059,6.862779e-06
59549,28,1997,3,67435.951719,59320.971209,91449.606255,0.000202,0.000178,2.434494e-05
63898,28,1998,5,68062.052369,59320.971209,91449.606255,0.000340,0.000297,4.370541e-05
68826,28,1999,3,68688.153018,59320.971209,91449.606255,0.000206,0.000178,2.810155e-05
73414,28,2000,4,69314.253668,59320.971209,91449.606255,0.000277,0.000237,3.997313e-05


In [35]:
raw_stats.head()

,Unnamed: 0,Unnamed: 0.1,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year,region,sub-region,intermediate-region,GCPNT_LAT,GCPNT_LON
0,0,0,1,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975],1983,Europe,Western Europe,Western Europe,48.138002,11.563435
1,1,2,763005,2784,Germany,1,1,40.658337,0.058337,0.058337,['2013.08.06'],[0.058337],[40.658337],2013,Europe,Western Europe,Western Europe,48.138002,11.563435
2,2,3,4,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318],1983,Europe,Western Europe,Western Europe,47.849642,12.102591
3,3,5,22703,2833,Germany,1,1,41.787350,1.187350,1.187350,['1984.07.11'],[1.18735],[41.78735],1984,Europe,Western Europe,Western Europe,47.849642,12.102591
4,4,6,349979,2833,Germany,1,1,41.393425,0.793425,0.793425,['1998.08.11'],[0.793425],[41.393425],1998,Europe,Western Europe,Western Europe,47.849642,12.102591


In [40]:
# Columns
columns = ['ID_HDC_G0','CTR_MN_NM', 'region', 'sub-region', 'intermediate-region', 'GCPNT_LAT', 'GCPNT_LON']
test3 = test2[0].merge(raw_stats[columns], on = 'ID_HDC_G0', how = 'inner')

In [41]:
test3.head()

,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop,CTR_MN_NM,region,sub-region,intermediate-region,GCPNT_LAT,GCPNT_LON
0,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0,United States,Americas,Northern America,Northern America,37.730079,-121.431413
1,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0,United States,Americas,Northern America,Northern America,37.730079,-121.431413
2,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0,United States,Americas,Northern America,Northern America,37.730079,-121.431413
3,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0,United States,Americas,Northern America,Northern America,37.730079,-121.431413
4,28,1983,1,59320.971209,59320.971209,91449.606255,0.000059,0.000059,0.0,United States,Americas,Northern America,Northern America,37.730079,-121.431413
